In [1]:
import sys

import numpy as np
import qiskit

from quantum_walks.state_circuit_generator import StateCircuitGenerator, MHSTreeGeneratorExhaustive, MergingStatesGenerator, MHSTreeGeneratorHeuristic, QiskitDefaultGenerator
from quantum_walks.utils.general import make_dict
from quantum_walks.utils.qiskit_utilities import remove_leading_cx_gates
from quantum_walks.utils.validation import execute_circuit, get_state_vector, get_fidelity

sys.executable

'/home/teague/Projects/python-envs/qwalks-env/bin/python3.11'

In [2]:
def check_fidelity(circuit: qiskit.QuantumCircuit, target_state: dict[str, complex], fidelity_tol: float = 1e-8):
    output_state_vector = execute_circuit(circuit)
    target_state_vector = get_state_vector(target_state)
    fidelity = get_fidelity(output_state_vector[:len(target_state_vector)], target_state_vector)
    assert abs(1 - fidelity) < fidelity_tol, f"Failed to prepare the state. Fidelity: {fidelity}"
    return True

In [12]:
def prepare_walk_circ(
    target_state: dict[str, complex],
    circuit_generator: StateCircuitGenerator,
    basis_gates: list[str],
    optimization_level: int,
    ensure_fidelity: bool = False,
    fidelity_tol: float = 1e-8
) -> qiskit.QuantumCircuit:
    circuit = circuit_generator.generate_circuit(target_state)
    circuit_transpiled = qiskit.transpile(circuit, **make_dict(basis_gates, optimization_level))
    circuit_transpiled = remove_leading_cx_gates(circuit_transpiled)

    if ensure_fidelity:
        check_fidelity(circuit_transpiled, target_state)

    return circuit_transpiled

In [11]:
def generate_state(num_terms):
    num_qubits = (num_terms).bit_length()
    coeffs = np.random.uniform(0, 1, size=num_terms)
    coeffs = np.sqrt(coeffs / np.sum(coeffs)).tolist()
    bitstrings = [bin(i)[2:].zfill(num_qubits)[::-1] for i in range(2**num_qubits)] # Little endian because Qiskit
    return {state: amplitude for state, amplitude in zip(bitstrings[:len(coeffs)], coeffs)}

def get_state_from_H(H):
    num_qubits = (len(H)).bit_length()
    coeffs = np.array([t[0] for t in H])
    coeffs = np.sqrt(coeffs / np.sum(coeffs)).tolist()
    bitstrings = [bin(i)[2:].zfill(num_qubits)[::-1] for i in range(2**num_qubits)] # Little endian because Qiskit
    return {state: amplitude for state, amplitude in zip(bitstrings[:len(coeffs)], coeffs)}

In [5]:
state = generate_state(10)
state

{'0000': 0.37344208617143704,
 '1000': 0.43808441609895077,
 '0100': 0.43702578967199646,
 '1100': 0.2542512212490401,
 '0010': 0.29647466478422074,
 '1010': 0.13691841247811493,
 '0110': 0.2613286512877809,
 '1110': 0.3016056003974321,
 '0001': 0.25218520115287263,
 '1001': 0.28894285232924843}

In [6]:
generators = [
    MHSTreeGeneratorExhaustive(change_basis=True, multiedge=True),
    MergingStatesGenerator(),
    MHSTreeGeneratorHeuristic(multiedge=False),
    QiskitDefaultGenerator()
]
prepare_circ = prepare_walk_circ(
    state,
    generators[0],
    basis_gates=["rx", "ry", "rz", "h", "cx"],
    optimization_level=3,
    ensure_fidelity=True
)
print(prepare_circ.count_ops())
prepare_circ.draw(fold=-1)

OrderedDict([('ry', 23), ('rz', 22), ('cx', 18), ('rx', 6)])


┌───────┐                   ┌───┐┌──────────┐┌──────────┐      ┌───────────┐                                  ┌─────────┐                                                                                                                                                                                                                                                                                                                                                                         ┌───────┐                                                                                                   
q_0: ─┤ Rx(π) ├───────────────────┤ X ├┤ Ry(-π/2) ├┤ Rz(-π/2) ├──■───┤ Ry(2.748) ├───────────────────────────────■──┤ Ry(π/2) ├──────────────────────────────────────────────────────────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────────────■─────┤ Rx(π) ├───────────────────────────────────────────────────────────────────────────────────────────────────
      └───────┘                   └─┬─┘└──────────┘└──────────┘  │   └───────────┘                               │  └─────────┘              ┌───┐                                               │                                             ┌───┐┌──────────┐                     ┌─────────┐  ┌────────────┐                     ┌────────┐    ┌──────────┐                                                  ┌───┐                                                       │                    │     └───────┘                                                                                                   
q_1: ───────────────────────────────┼────────────────────────────┼───────────────────────────────────────────────┼───────────────────────────┤ X ├───────────────────────────────────────────────┼─────────────────────────────────────────────┤ X ├┤ Ry(-π/2) ├────────────────■────┤ Rz(π/2) ├──┤ Ry(3.1058) ├────────────────■────┤ Rz(-π) ├────┤ Ry(-π/2) ├──────────────────────────────────────────────────┤ X ├───────■────────────────────────────■──────────────────┼────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     ┌──────────┐┌───────────────┐  │                            │                                               │                           └─┬─┘                                               │                                             └─┬─┘└──┬───┬───┘  ┌────────┐    │    └─────────┘  └────────────┘                │    └────────┘    └──────────┘                                                  └─┬─┘┌───┐  │    ┌─────────┐  ┌───────┐  │                  │                    │                                       ┌────────────┐                                 ┌──────────┐  ┌─────────┐  
q_2: ┤ Rz(-π/2) ├┤ Ry(0.0089463) ├──■────────────────────────────┼───────────────────────────────────────────────┼─────────────────────────────┼─────────────────────────────────────────────────┼───────────────────────────────────────────────┼─────┤ X ├──────┤ Rz(-π) ├────┼───────────────────────────────────────────────┼───────────────────────────────────────────────■──────────────────────────────────┼──┤ X ├──┼────┤ Ry(π/2) ├──┤ Rz(π) ├──┼──────────────────┼────────────────────┼───────────────────────────────────■───┤ Ry(1.9088) ├──────────────────────────────■──┤ Rz(-π/2) ├──┤ Ry(π/2) ├──
     └┬────────┬┘└┬──────────────┤                             ┌─┴─┐┌─────────────┐┌────────────┐┌────────────┐┌─┴─┐┌─────────┐┌────────────┐  │  ┌──────────────┐┌────────────┐┌─────────────┐┌─┴─┐┌────────────┐┌────────────┐┌─────────────┐  │     └─┬─┘    ┌─┴────────┴─┐┌─┴─┐┌─────────────┐┌────────────┐┌────────────┐┌─┴─┐┌────────────┐┌─────────────┐┌────────────┐┌─┴─┐┌─────────────┐┌─────────────┐  │  └─┬─┘┌─┴─┐┌─┴─────

In [13]:
def construct_select_circ(num_ham_qubits: int, H): # Haven't decided best way to represent H yet
    num_ctrl_qubits = (len(H)).bit_length()
    ctrl_qubits = [i for i in range(num_ctrl_qubits)]
    bitstrings = [bin(i)[2:].zfill(num_ctrl_qubits)[::-1] for i in range(2**num_ctrl_qubits)] # Little endian because Qiskit
    circ = qiskit.QuantumCircuit(num_ctrl_qubits + num_ham_qubits)
    for ctrl_bitstring, pauli_term in zip(bitstrings[:len(H)], H):
        #print(ctrl_bitstring, pauli_term)
        pauli_str = "".join([t[0] for t in pauli_term]) # The Pauli is always the first character
        qubit_idx = [num_ctrl_qubits + int(t[1:]) for t in pauli_term] # The qubit index is always everything after, offset the indices to account for the other register
        #print(pauli_str, qubit_idx)
        ctrl_pauli_gate = qiskit.circuit.library.PauliGate(pauli_str).control(num_ctrl_qubits=num_ctrl_qubits, ctrl_state=ctrl_bitstring)
        circ.append(ctrl_pauli_gate, ctrl_qubits + qubit_idx)
    return circ

In [9]:
H = [
    ('Z0',),
    ('X1',),
    ('Y0',),
    ('Z1',),
    ('X0', 'Z1'),
    ('Y0', 'Y1'),
    ('Z0', 'Z1'),
    ('X0', 'X1'),
    ('Z0', 'Y1'),
    ('Y1',),
]
select_circ = construct_select_circ(2, H)
select_circ.draw(fold=-1)

q_0: ─────o───────────o───────────o───────────o────────────o─────────────o─────────────o─────────────o─────────────■────────────■──────
          │           │           │           │            │             │             │             │             │            │      
q_1: ─────o───────────o───────────o───────────o────────────■─────────────■─────────────■─────────────■─────────────o────────────o──────
          │           │           │           │            │             │             │             │             │            │      
q_2: ─────o───────────o───────────■───────────■────────────o─────────────o─────────────■─────────────■─────────────o────────────o──────
          │           │           │           │            │             │             │             │             │            │      
q_3: ─────o───────────■───────────o───────────■────────────o─────────────■─────────────o─────────────■─────────────o────────────■──────
     ┌────┴─────┐     │      ┌────┴─────┐     │      ┌─────┴──────┐┌─────┴──────┐┌─────┴──────┐┌─────┴──────┐┌─────┴──────┐     │      
q_4: ┤ Pauli(Z) ├─────┼──────┤ Pauli(Y) ├─────┼──────┤0           ├┤0           ├┤0           ├┤0           ├┤0           ├─────┼──────
     └──────────┘┌────┴─────┐└──────────┘┌────┴─────┐│  Pauli(XZ) ││  Pauli(YY) ││  Pauli(ZZ) ││  Pauli(XX) ││  Pauli(ZY) │┌────┴─────┐
q_5: ────────────┤ Pauli(X) ├────────────┤ Pauli(Z) ├┤1           ├┤1           ├┤1           ├┤1           ├┤1           ├┤ Pauli(Y) ├
                 └──────────┘            └──────────┘└────────────┘└────────────┘└────────────┘└────────────┘└────────────┘└──────────┘

In [8]:
def construct_qubitization_circ(num_qubits, hamiltonian):
    generators = [
        MHSTreeGeneratorExhaustive(change_basis=True, multiedge=True),
        MergingStatesGenerator(),
        MHSTreeGeneratorHeuristic(multiedge=False),
        QiskitDefaultGenerator()
    ]
    
    prepare_circ = prepare_walk_circ(
        get_state_from_H(hamiltonian),
        generators[0],
        basis_gates=["rx", "ry", "rz", "h", "cx"],
        optimization_level=3,
        ensure_fidelity=True
    )
    # Remove the coefficients from H now
    H = [t[1] for t in hamiltonian]
    select_circ = construct_select_circ(num_qubits, H)

    qubitization_circ = qiskit.QuantumCircuit(num_qubits)
    # Put the PREPARE op on top of the circuit
    qubitization_circ.tensor(prepare_circ, inplace=True)
    prepare_op = qubitization_circ.copy()
    # Stick the SELECT op in
    qubitization_circ.compose(select_circ, qubits=qubitization_circ.qubits, inplace=True)
    # Finish with PREPARE^dag
    qubitization_circ.compose(prepare_op.inverse(), qubits=qubitization_circ.qubits, inplace=True)

    return qubitization_circ

In [11]:
H = [
    ('Z0',),
    ('X1',),
    ('Y0',),
    ('Z1',),
    ('X0', 'Z1'),
    ('Y0', 'Y1'),
    ('Z0', 'Z1'),
    ('X0', 'X1'),
    ('Z0', 'Y1'),
    ('Y1',),
]
qubitization_circ = construct_qubitization_circ(2, H)
qubitization_circ.draw(fold=-1)

┌───────┐                   ┌───┐┌─────────┐       ┌─────────┐  ┌────────────┐                   ┌─────────┐                                                                                                                                                                                                                                                                                                                                                                   ┌───────┐                                                                                                end                                                                                                                                    end   ┌────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                ┌──────────┐                                   ┌─────────────┐  ┌──────────┐                    ┌──────────┐     ┌───┐         ┌────────┐              
q_0: ────┤ Rx(π) ├───────────────────┤ X ├┤ Ry(π/2) ├──■────┤ Rz(π/2) ├──┤ Ry(2.5288) ├────────────────■──┤ Ry(π/2) ├───────────────────────────────────■────────────────────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────■────┤ Rx(π) ├─────────────────────────────────────────────────────────────────────────────────────────────────░───────o───────────o───────────o───────────o────────────o─────────────o─────────────o─────────────o─────────────■────────────■────────░────┤ Rx(-π) ├───────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────■──┤ Ry(-π/2) ├────────────────────────────────■──┤ Ry(-2.5288) ├──┤ Rz(-π/2) ├─────────────────■──┤ Ry(-π/2) ├─────┤ X ├─────────┤ Rx(-π) ├──────────────
         └───────┘                   └─┬─┘└─────────┘  │    └─────────┘  └────────────┘                │  └─────────┘                ┌───┐              │                    │                            ┌───┐┌──────────┐ ┌──────────┐        ┌─────────┐  ┌────────────┐                   ┌─────────┐ ┌──────────┐                                                         ┌───┐┌────────┐                                                                 │                   │    └───────┘                                                                                                 ░       │           │           │           │            │             │             │             │             │            │        ░    └────────┘                                                                                                   │                  │                                       ┌───────┐                      ┌───┐ ┌─────────┐     ┌──────────┐                                                            ┌─────────────┐  ┌──────────┐                      ┌─────────┐  ┌─────────┐┌───┐                              │                   │  └──────────┘┌───┐                           │  └─────────────┘  └──────────┘                 │  └──────────┘     └─┬─┘         └────────┘              
q_1: ──────────────────────────────────┼───────────────┼───────────────────────────────────────────────┼─────────────────

In [15]:
H = [
    (1, ('Z0',)),
    (0.5, ('X0',)),
    (0.2, ('Y0',)),
]
qubitization_circ = construct_qubitization_circ(1, H)
qubitization_circ.draw(fold=-1)

┌───────────┐  ┌──────────┐                      ┌─────────┐  ┌────────────┐                   ┌─────────┐ ┌──────────┐                                       ┌─────────┐  ┌──────────┐     ┌─────────────┐  ┌──────────┐                      ┌─────────┐    ┌──────────┐               
q_0: ─┤ Rz(-3π/4) ├──┤ Rx(-π/2) ├─────────────────■────┤ Rz(π/2) ├──┤ Ry(2.1863) ├────────────────■──┤ Rx(π/2) ├─┤ Rz(-π/4) ├──────o───────────o───────────■────────┤ Rz(π/4) ├──┤ Rx(-π/2) ├──■──┤ Ry(-2.1863) ├──┤ Rz(-π/2) ├─────────────────■────┤ Rx(π/2) ├────┤ Rz(3π/4) ├───────────────
     ┌┴───────────┴┐┌┴──────────┴┐┌────────────┐┌─┴─┐┌─┴─────────┴─┐├────────────┤┌────────────┐┌─┴─┐├─────────┤┌┴──────────┴┐     │           │           │      ┌─┴─────────┴─┐├──────────┤┌─┴─┐├─────────────┤┌─┴──────────┴┐┌────────────┐┌─┴─┐┌─┴─────────┴─┐┌─┴──────────┴┐┌────────────┐
q_1: ┤ Rz(-1.1465) ├┤ Ry(1.1541) ├┤ Rz(2.0924) ├┤ X ├┤ Rz(-2.8185) ├┤ Ry(2.0383) ├┤ Rz(2.2097) ├┤ X ├┤ Rz(π/2) ├┤ Ry(1.1769) ├─────o───────────■───────────o──────┤ Ry(-1.1769) ├┤ Rz(-π/2) ├┤ X ├┤ Rz(-2.2097) ├┤ Ry(-2.0383) ├┤ Rz(2.8185) ├┤ X ├┤ Rz(-2.0924) ├┤ Ry(-1.1541) ├┤ Rz(1.1465) ├
     └─────────────┘└────────────┘└────────────┘└───┘└─────────────┘└────────────┘└────────────┘└───┘└─────────┘└────────────┘┌────┴─────┐┌────┴─────┐┌────┴─────┐└─────────────┘└──────────┘└───┘└─────────────┘└─────────────┘└────────────┘└───┘└─────────────┘└─────────────┘└────────────┘
q_2: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ Pauli(Z) ├┤ Pauli(X) ├┤ Pauli(Y) ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                              └──────────┘└──────────┘└──────────┘